# Set-up

In [ ]:
import h5py
from scipy.sparse import csr_matrix
from scipy.io import mmwrite
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
from tqdm.auto import tqdm

indir = '/projects_ng/SC_DATA/LinnarsonBraun/'
outdir = './data/'

/projects_ng/environments/EWCE2024/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_url = 'https://storage.googleapis.com/linnarsson-lab-human/HumanFetalBrainPool.h5'
data_fname = f"{indir}HumanFetalBrainPool.h5"

!mkdir -p $(dirname {data_fname})
!wget -nc -O {data_fname} {data_url}

File ‘/projects_ng/SC_DATA/LinnarsonBraun/HumanFetalBrainPool.h5’ already there; not retrieving.


In [ ]:
data_url = 'https://storage.googleapis.com/linnarsson-lab-human/human_dev.h5ad'
data_fname = f"{indir}human_dev.h5ad"


!mkdir -p $(dirname {data_fname})
!wget -nc -O {data_fname} {data_url}

File ‘/projects_ng/SC_DATA/LinnarsonBraun/human_dev.h5ad’ already there; not retrieving.


# **Part 1: Load the data** `.h5 -> .csv + .mtx`

In [4]:
filename =  f"{indir}HumanFetalBrainPool.h5"
f = h5py.File(filename, "r")


In [5]:
atts = pd.DataFrame([{k: f['shoji'][k].shape for k in f['shoji'].keys()}]).T.reset_index().rename({'index': 'key', 0: 'dims'}, axis=1)
atts = atts.sort_values('dims')

In [6]:
dim_names = {
    '51': 'annotation',
    '617': 'clusters',
    '59480': 'genes',
    '1665937': 'ncells'
}
a = atts['dims'].astype(str)
for s,r in dim_names.items():
    a = a.str.replace(s,r)
atts['dim_names'] = a
att_groups = {k: v.values for k, v in atts.groupby('dim_names')['key']}


In [7]:
# factors and embeddings:
dfs = dict(
    Embedding = f['shoji'][att_groups['(ncells, 2)'][0]][()],
    Factors = f['shoji'][att_groups['(ncells, 50)'][0]][()],
    Loadings = f['shoji'][att_groups['(genes, 50)'][0]][()])

In [8]:
# Clusters x annotations
dfs.update(dict(ClusterAnnotationPosterior = f['shoji'][att_groups['(clusters, annotation)'][0]][()]))

In [9]:
# Clusters x genes
dfs.update({f'ClusterGenes.{k}': f['shoji'][k][()] for k in att_groups['(clusters, genes)']})

In [10]:
# AnnotationInfo
AnnotationInfo = {k: f['shoji'][k][()] for k in att_groups['(annotation,)']}
dfs.update(dict(AnnotationInfo = pd.DataFrame(AnnotationInfo).apply(lambda x: x.str.decode("utf-8") , 0)))

In [29]:
# CellInfo
CellInfo = {k: f['shoji'][k][()] for k in att_groups['(ncells,)']}
dfs.update(dict(CellInfo = pd.DataFrame(CellInfo).apply(lambda x: x if not x.dtype == 'O' else x.str.decode("utf-8") , 0)))

In [12]:
# ClusterInfo
ClusterInfo = {k: f['shoji'][k][()] for k in att_groups['(clusters,)']}
dfs.update(dict(ClusterInfo = pd.DataFrame(ClusterInfo).apply(lambda x: x if not x.dtype == 'O' else x.str.decode("utf-8") , 0)))

In [13]:
# GeneInfo
GeneInfo = {k: f['shoji'][k][()] for k in att_groups['(genes,)']}
dfs.update(dict(GeneInfo = pd.DataFrame(GeneInfo).apply(lambda x: x if not x.dtype == 'O' else x.str.decode("utf-8") , 0)))

In [120]:
expression_is_loaded = False

In [ ]:
# Expression
Expression = csr_matrix(f['shoji'][att_groups['(ncells, genes)'][0]][()])
expression_is_loaded = True

In [ ]:
f.close()

## Save the dataframes

In [14]:
for k, df in tqdm(dfs.items()):
    if isinstance(df, pd.DataFrame):
        df.to_csv(f'{outdir}{k}.csv')
    else:
        np.savetxt(f'{outdir}{k}.csv', df, delimiter=',')

100%|██████████| 12/12 [02:04<00:00, 10.39s/it]


## Save the expression matrix

In [ ]:
mmwrite(f'{outdir}Expression_ValidGenes.sparseMatrix.mtx', 
        Expression[:,dfs['GeneInfo']['ValidGenes']], 
        comment = 'Expression_ValidGenes')

mmwrite(f'{outdir}Expression.sparseMatrix.mtx', 
        Expression, 
        comment = 'Expression')

# **Part 2: Update the annotation**  
## `Run after processing the lineage annotation` 

In [71]:
cellmeta = dfs['CellInfo'].copy()

In [73]:
new_anno = pd.read_excel('./cluster_lineage_annotation.summary.xlsx')

In [74]:
cellmeta = cellmeta.merge(new_anno[['ClusterID.(PoolClean)', 'LongName', 'ShortName', 'discarded']], 
                          left_on='Clusters', right_on = 'ClusterID.(PoolClean)', sort=False)

cellmeta.to_csv(f'{outdir}CellInfo.annotated.csv')

# **Part 3: Save the subsets for analysis**  

In [86]:
import time


In [88]:
cellmeta.Age.nunique() == cellmeta.Age.round(2).nunique()
cellmeta.Age = cellmeta.Age.round(2).astype(str)

cellmeta.Subregion = cellmeta.Subregion.str.replace(' ', '_')

In [ ]:
# we want to analyse the data by subregion and timepoint, and then measure celltype differences.
# celltypes that don't have enough cells are not included
# we also save the cell metadata of the subset to keep things clear
# And we don't include the discarded cells
# And we want to include only the selected regions 

selected_regions = ['Brain', 'Head', 'Forebrain', 'Telencephalon']
group_vars = ['Subregion', 'Age']
celltype_var = ['ShortName']
enough_cells = 100

meta = cellmeta.copy()
meta.discarded = meta.discarded | ~meta.Region.isin(selected_regions)

pbar = tqdm(total = (~meta[group_vars].duplicated()).sum())

for region, rmeta in meta.groupby(group_vars[0]):

    for age, ameta in rmeta.groupby(group_vars[1]):

        t0 = time.time()

        if not ameta.discarded.all():
                                    
            cinfo = ameta[celltype_var].value_counts().reset_index()
            cinfo['include'] = cinfo.iloc[:,-1] >= enough_cells
        
            included = cinfo.iloc[:,0][cinfo['include'] & ~cinfo.iloc[:,0].isna()].values
    
            selmeta = pd.concat([
                ~meta.discarded,
                meta[celltype_var].isin(included),
                meta[group_vars[0]] == region,
                meta[group_vars[1]] == age], axis=1).all(1)
    
            print(f'[{region}][{age}]')
    
            if (selmeta.sum() > 0):
    
                path = f'{outdir}/by_{group_vars[0]}/{region}/by_{group_vars[1]}/{age}/'
                !mkdir -p {path}
                
                print(f'''\
    - Starting {ameta.shape[0]} cells
    - Selected {selmeta.sum()} cells
    - Included cell types: {", ".join(included)}
    - Excluded cell types: {", ".join(cinfo.iloc[:,0][~cinfo.iloc[:,0].isin(included)])}
    -> Saving to {path}...
    ''')
    
                print('...Saving metadata')
                meta[selmeta].to_csv(f'{path}cell_meta.csv')

                
                print('...Getting expression')
                if not expression_is_loaded:
                    exp = csr_matrix(f['shoji'][att_groups['(ncells, genes)'][0]][selmeta.to_numpy(),:])
                else:
                    exp = Expression[selmeta,:]

                print('...Saving expression')
                mmwrite(f'{path}Expression.sparseMatrix.mtx', 
                        exp, 
                        comment = 'Expression')
                
                print(f'-> Done in {round(time.time()-t0,2)} seconds!#########################\n')
            else:
                print('-> Not saving\n')
    
            
        pbar.update(1)




100%|██████████| 84/84 [05:17<00:00,  3.78s/it]


[Brain][6.0]
    - Starting 85999 cells
    - Selected 85160 cells
    - Included cell types: RGC, ExNeuBl, ExNeu, InNeu, InNeuBl, ExNeuIPC, InNeuIPC, Imm, Pery
    - Excluded cell types: Endo, Eryt, OPC, GlioBl, COPs, Fibr
    -> Saving to ./data//by_Subregion/Brain/by_Age/6.0/...
    
...Saving metadata
...Getting expression
...Saving expression



  2%|▏         | 2/84 [05:15<2:57:28, 129.86s/it]

-> Done in 314.65 seconds!#########################

[Cortex][10.0]
    - Starting 51372 cells
    - Selected 51210 cells
    - Included cell types: ExNeu, RGC, ExNeuBl, ExNeuIPC, GlioBl, InNeu, Imm
    - Excluded cell types: Pery, Eryt, OPC, Endo, COPs, VSMC, InNeuIPC
    -> Saving to ./data//by_Subregion/Cortex/by_Age/10.0/...
    
...Saving metadata
...Getting expression
...Saving expression



 13%|█▎        | 11/84 [08:11<39:28, 32.44s/it]  

...

-> Done in 72.26 seconds!#########################

[Subcortex][9.5]
    - Starting 12859 cells
    - Selected 12681 cells
    - Included cell types: InNeu, ExNeu, GlioBl, InNeuIPC, RGC, ExNeuBl, ExNeuIPC
    - Excluded cell types: Eryt, Imm, OPC, Pery, Endo, COPs, VSMC
    -> Saving to ./data//by_Subregion/Subcortex/by_Age/9.5/...
    
...Saving metadata
...Getting expression
...Saving expression



 93%|█████████▎| 78/84 [51:27<04:49, 48.24s/it]

-> Done in 53.81 seconds!#########################

[Telencephalon][6.7]
    - Starting 7052 cells
    - Selected 6903 cells
    - Included cell types: RGC, InNeu, ExNeu, ExNeuBl, ExNeuIPC, InNeuIPC
    - Excluded cell types: GlioBl, Imm, Eryt, Pery, Endo
    -> Saving to ./data//by_Subregion/Telencephalon/by_Age/6.7/...
    
...Saving metadata
...Getting expression
...Saving expression



 94%|█████████▍| 79/84 [52:07<03:52, 46.44s/it]

-> Done in 39.93 seconds!#########################



In [122]:
print('Done')

Done
